## A.I. Assignment 5

## Learning Goals

By the end of this lab, you should be able to:
* Get more familiar with tensors in pytorch 
* Create a simple multilayer perceptron model with pytorch
* Visualise the parameters


### Task

Build a fully connected feed forward network that adds two bits. Determine the a propper achitecture for this network (what database you use for this problem? how many layers? how many neurons on each layer? what is the activation function? what is the loss function? etc)

Create at least 3 such networks and compare their performance (how accurate they are?, how farst they are trained to get at 1 accuracy?)

Display for the best one the weights for each layer.


In [1]:
import torch
import torch.nn as nn
from collections import OrderedDict


In [ ]:
model = nn.Sequential(OrderedDict([
    ('hidden', nn.Linear(2, 2)),
    ('relu', nn.ReLU()),
    ('output', nn.Linear(2, 2)),
]))

In [22]:
print(model)

Sequential(
  (hidden): Linear(in_features=2, out_features=2, bias=True)
  (relu): ReLU()
  (output): Linear(in_features=2, out_features=2, bias=True)
)


In [ ]:
data_in = torch.tensor([[0,0], [0,1], [1,0], [1,1]], dtype=torch.float32)

print(data_in)

tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]])


In [ ]:
data_target = torch.tensor([[0, 0], [1, 0], [1, 0], [0, 1]], dtype=torch.float32)

print(data_target)

tensor([[0., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.]])


In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [ ]:
for epoch in range(10000):
    y_pred = model(data_in)
    loss = criterion(y_pred, data_target)
    print('epoch: ', epoch,' loss: ', loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


epoch:  0  loss:  0.30428630113601685
epoch:  1  loss:  0.21473819017410278
epoch:  2  loss:  0.1732952892780304
epoch:  3  loss:  0.16880087554454803
epoch:  4  loss:  0.1760648787021637
epoch:  5  loss:  0.18311995267868042
epoch:  6  loss:  0.180905282497406
epoch:  7  loss:  0.17420823872089386
epoch:  8  loss:  0.16418935358524323
epoch:  9  loss:  0.15722639858722687
epoch:  10  loss:  0.15278545022010803
epoch:  11  loss:  0.14874126017093658
epoch:  12  loss:  0.1453964114189148
epoch:  13  loss:  0.14133280515670776
epoch:  14  loss:  0.13589052855968475
epoch:  15  loss:  0.1303672045469284
epoch:  16  loss:  0.12477793544530869
epoch:  17  loss:  0.11931723356246948
epoch:  18  loss:  0.11451120674610138
epoch:  19  loss:  0.10962740331888199
epoch:  20  loss:  0.10622429102659225
epoch:  21  loss:  0.10306400060653687
epoch:  22  loss:  0.10049011558294296
epoch:  23  loss:  0.09861189872026443
epoch:  24  loss:  0.09724270552396774
epoch:  25  loss:  0.09578844904899597
ep

In [ ]:
print(model(data_in))

tensor([[6.6667e-01, 3.4335e-10],
        [6.6667e-01, 3.4335e-10],
        [6.6667e-01, 3.4335e-10],
        [2.3842e-07, 1.0000e+00]], grad_fn=<AddmmBackward0>)


In [ ]:
print(model.state_dict())

OrderedDict({'hidden.weight': tensor([[-0.6869, -0.3976],
        [ 1.7546,  1.6966]]), 'hidden.bias': tensor([-0.8990, -1.9832]), 'output.weight': tensor([[ 0.5651, -0.4541],
        [-0.1611,  0.6812]]), 'output.bias': tensor([6.6667e-01, 3.4335e-10])})


In [ ]:
accuracy = 0
for i in range(len(data_target)):
    difference = (model(data_in)[i] - data_target[i]).abs()
    if torch.all(difference < 0.5):
        accuracy += 1

print(f'Accuracy: {accuracy/len(data_target) * 100}%')

Accuracy: 75.0%


In [ ]:
import torch
import torch.nn as nn
from collections import OrderedDict

def train_model(model, criterion, optimizer, X_train, y_train, num_epochs=10000, tolerance=1e-3):
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()

        if loss.item() < tolerance:
            break

        if (epoch + 1) % 100 == 0:
            print(f'Epoch {epoch + 1}/{num_epochs} | Loss: {loss.item()}')

    return model, epoch + 1

def evaluate_model(model, X, y):
    with torch.no_grad():
        predicted = torch.clamp(model(X).round(), 0, 1)
        accuracy = (predicted == y).all(dim=1).float().mean().item() * 100
    return accuracy

# Dataset definition
X_train = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)  
y_train = torch.tensor([[0, 0], [0, 1], [0, 1], [1, 0]], dtype=torch.float32)

# Architecture definition for 3 models
architectures = [
    OrderedDict([
        ('hidden1', nn.Linear(2, 5)),
        ('activation1', nn.ReLU()),
        #('output', nn.Linear(5, 2)),
        #('activation2', nn.LeakyReLU())
        ('hidden2', nn.Linear(5,8)),
        ('activation2', nn.LeakyReLU()),
        ('output', nn.Linear(8, 2)),
        ('activation3', nn.ReLU())
    ]),
    OrderedDict([
        ('hidden1', nn.Linear(2, 3)),
        ('activation1', nn.ReLU()),
        ('output', nn.Linear(3, 2)),
        ('activation2', nn.Tanh())
    ]),
    OrderedDict([
        ('hidden1', nn.Linear(2, 3)),
        ('activation1', nn.ReLU()),
        ('output', nn.Linear(3, 2)),
        ('activation2', nn.Sigmoid())
    ])
]

# Training config
learning_rates = [0.01, 0.75, 0.2]
num_epochs = 10000

# Train and evaluate 
best_accuracy = 0
best_model = None
best_epoch = 0
best_idx = 0
for idx, (architecture, learning_rate) in enumerate(zip(architectures, learning_rates)):
    model = nn.Sequential(architecture)
    criterion = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    # Tried ADAM -> probably good for more complex models, not so much for this
    trained_model, epoch = train_model(model, criterion, optimizer, X_train, y_train, num_epochs=num_epochs)
    accuracy = evaluate_model(trained_model, X_train, y_train)
    print(f'Model {idx+1} Accuracy: {accuracy}%')

    if accuracy >=  best_accuracy:
        if accuracy == best_accuracy:
            best_accuracy = accuracy
            best_model = trained_model
            best_epoch = epoch
            best_idx = idx
        else:
            best_accuracy = accuracy
            best_model = trained_model
            best_epoch = epoch
            best_idx = idx

# Display the weights for each layer of the best model
print("\nWeights for the best model:")
for name, param in best_model.named_parameters():
    print(name, param.data)

print(f'Best model: {best_idx + 1} with accuracy {best_accuracy}% and trained for {best_epoch} epochs')

Epoch 100/10000 | Loss: 0.3437122404575348
Epoch 200/10000 | Loss: 0.342906653881073
Epoch 300/10000 | Loss: 0.3422850966453552
Epoch 400/10000 | Loss: 0.34164807200431824
Epoch 500/10000 | Loss: 0.3409689664840698
Epoch 600/10000 | Loss: 0.3402280807495117
Epoch 700/10000 | Loss: 0.3394036591053009
Epoch 800/10000 | Loss: 0.3384491205215454
Epoch 900/10000 | Loss: 0.33715924620628357
Epoch 1000/10000 | Loss: 0.3356974720954895
Epoch 1100/10000 | Loss: 0.33389800786972046
Epoch 1200/10000 | Loss: 0.3316112160682678
Epoch 1300/10000 | Loss: 0.3292413651943207
Epoch 1400/10000 | Loss: 0.3263464868068695
Epoch 1500/10000 | Loss: 0.3227626085281372
Epoch 1600/10000 | Loss: 0.31907108426094055
Epoch 1700/10000 | Loss: 0.31550025939941406
Epoch 1800/10000 | Loss: 0.3115030825138092
Epoch 1900/10000 | Loss: 0.30702972412109375
Epoch 2000/10000 | Loss: 0.3022281229496002
Epoch 2100/10000 | Loss: 0.29706186056137085
Epoch 2200/10000 | Loss: 0.29167789220809937
Epoch 2300/10000 | Loss: 0.2861505